In [1]:
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import numpy as np
import os
import cv2
import time
import pandas as pd

def load_images():
    dir = os.path.join(os.sep,'home','mohit','Desktop','dataset')
    images = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.imread(os.path.join(dir,'Cheque'+str(k),i,i+'_'+j+'.tif'))
                    images['c'+str(k)+'_'+i+'_'+j] = cv2.cvtColor(images['c'+str(k)+'_'+i+'_'+j], cv2.COLOR_BGR2YCrCb)
                except:
                    break
    return images

def n_size(images):
    size = {} #no. of ink pixels .. considering all non white pixels(<th) as ink pixels
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                count = 0
                try:
                    image = images['c'+str(k)+'_'+i+'_'+j][:,:,0]
                    for a in range(0,image.shape[0]):
                        for b in range(0,image.shape[1]):
                            if image[a][b] != 255:
                                count+=1
                    size['c'+str(k)+'_'+i+'_'+j] = count
                   # print('c'+str(k)+'_'+i+'_'+j,count)
                except:
                    #print('..')
                    break  
    return size




In [2]:
def features(img1,img2,s):
    fv1,fv2,fv3,fv4,fv5 = 0,0,0,0,0
    for x in range(0, len(img1)):
        for y in range(0, len(img1[0])):
            if(img1[x,y]!=255):
                fv1+=img2[x,y]
    fv1/=s        
    for x in range(0, len(img1)):
        for y in range(0, len(img1[0])):
            if(img1[x,y]!=255):    
                fv2+=(img2[x,y]-fv1)**2
                fv5+= abs(img2[x,y]-fv1)
    fv2/=(s-1)
    fv5/=s
    std = fv2**0.5
    for x in range(0, len(img1)):
        for y in range(0, len(img1[0])):
            if(img1[x,y]!=255): 
                fv3+=((img2[x,y]-fv1)/std)**3
                fv4+=((img2[x,y]-fv1)/std)**4
    fv3/=s
    fv4/=s
    
    return np.array([fv1,fv2,fv3,fv4-3,fv5])

In [3]:
def statistical_features(images,sizes):
    hist = {}
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist1 = features(images['c'+str(k)+'_'+i+'_'+j][:,:,0],images['c'+str(k)+'_'+i+'_'+j][:,:,0],sizes['c'+str(k)+'_'+i+'_'+j])
                    hist2 = features(images['c'+str(k)+'_'+i+'_'+j][:,:,0],images['c'+str(k)+'_'+i+'_'+j][:,:,1],sizes['c'+str(k)+'_'+i+'_'+j])
                    hist3 = features(images['c'+str(k)+'_'+i+'_'+j][:,:,0],images['c'+str(k)+'_'+i+'_'+j][:,:,2],sizes['c'+str(k)+'_'+i+'_'+j])
                    hist['c'+str(k)+'_'+i+'_'+j]=np.concatenate([hist1,hist2,hist3])
                except:
                    break
                   
    return hist

In [4]:
images = load_images()
sizes = n_size(images)

In [5]:
hist = statistical_features(images,sizes)

In [6]:
import pandas as pd
features = pd.DataFrame(hist)
features

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,72.486643,75.916715,68.991495,73.673966,47.518584,45.440278,43.909430,43.503347,42.037456,41.841573,...,44.992625,91.557559,81.518617,80.132029,77.692927,76.910283,79.434095,79.408792,67.531250,70.691566
1,765.393218,799.214324,902.992835,802.165650,1385.684587,1495.572901,1513.454479,1331.520477,1326.773772,1337.934415,...,1469.893673,695.922030,834.776766,1196.972766,1119.432694,1243.583183,1018.327864,1082.956446,1295.147037,1110.813629
2,0.357176,0.262596,0.270734,0.264057,0.895227,0.875544,0.919442,0.937158,0.987267,0.993418,...,0.900352,-0.024418,0.147969,-0.117934,-0.000721,-0.074438,0.014334,-0.023388,0.251957,0.320553
3,0.053515,-0.078647,-0.169420,0.016253,-0.154238,-0.267029,-0.211859,0.018091,0.111941,0.067007,...,-0.253591,-0.930820,-0.902837,-1.057967,-1.015056,-1.040785,-0.938234,-0.961362,-0.919340,-0.886593
4,21.445255,22.008475,23.449644,21.966324,30.202374,31.809846,31.861701,29.375732,29.237657,29.256600,...,31.368488,22.265254,24.395739,29.712615,28.425790,30.113940,26.917004,27.824239,29.965069,28.021549
5,134.569493,134.792074,134.547838,134.412409,128.239646,128.162350,128.126265,129.474020,129.556769,129.348315,...,128.227139,119.373093,118.459441,119.431133,119.322599,120.026528,119.495606,119.349602,115.936298,113.386747
6,5.202663,5.121110,4.540086,6.103673,9.377748,10.801938,11.404624,5.940731,7.874508,6.254101,...,8.766082,18.228664,15.014987,26.844152,33.318770,33.229171,29.806985,27.231614,21.751653,12.422886
7,-0.204048,-0.271010,-0.123226,-0.224778,0.595813,0.655612,0.521652,0.538754,0.206781,0.578907,...,0.378935,0.596352,0.643477,0.941066,0.822276,0.671769,0.724408,0.789896,0.775411,0.654900
8,0.879730,0.698621,0.379037,1.107879,1.236524,1.468580,1.590188,0.929801,0.630401,2.098322,...,0.848292,0.320494,0.847565,1.801602,0.528672,0.398216,0.475170,1.089157,0.118908,1.054353
9,1.782412,1.784436,1.686243,1.881405,2.282802,2.412808,2.436413,1.899065,2.200704,1.928342,...,2.240056,3.290983,2.947976,3.866013,4.506914,4.521336,4.163394,3.895363,3.773617,2.710416


In [7]:
features.to_csv('/home/mohit/Desktop/featuresUpdated/statistical_features.csv',sep = ',')